# SET 4

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

featuresdf=pd.read_csv('./csv/set4.csv', index_col=0)
featuresdf = featuresdf.reset_index()
set4fake = featuresdf.copy()
for column in ['acceleration_x','acceleration_y','acceleration_z', 'gyro_x', 'gyro_y', 'gyro_z','second', 'Speed']:
    justnullsdf = set4fake[set4fake[column].isnull()]
    bookingids = justnullsdf['bookingID'].unique() # get all the bookingids that have null values inside them
    rows = 0
    for i in range (len(bookingids)):
        bookingidx = set4fake[set4fake['bookingID']==bookingids[i]]
        if bookingidx[column].isnull().all().all():
            set4fake = set4fake.drop(set4fake[set4fake.bookingID == bookingids[i]].index)
            
uniqueValues = set4fake['bookingID'].unique()
uniqueValues = np.sort(uniqueValues)
bookingids=featuresdf['bookingID'].unique()


In [ ]:
print(set4fake.shape)
set4_1fake = set4fake[set4fake['bookingID'].isin(uniqueValues[0:1650])]
print(set4_1fake.shape)
set4_2fake = set4fake[set4fake['bookingID'].isin(uniqueValues[1650:2900])]
print(set4_2fake.shape)
set4_3fake = set4fake[set4fake['bookingID'].isin(uniqueValues[2900:])]
print(set4_3fake.shape)
print(set4_1fake.shape[0] + set4_2fake.shape[0] + set4_3fake.shape[0])

(188106, 12)
(66258, 12)
(50640, 12)
(71208, 12)
188106


# SET 4-3

In [ ]:
# THIS IS ONLY FOR SET 3_3fake acceleration_x
justnullsdf = set4_3fake[set4_3fake['acceleration_x'].isnull()]
bookingids = justnullsdf['bookingID'].unique() # get all the bookingids that have null values inside them
for bookingid in bookingids: #loop for each bookingid so that we are dealing with the null values bookingid by bookingid which is easier to work with
    bookingidx = set4_3fake[set4_3fake['bookingID']==bookingid] # making a datfarame with bookingID = x
    bookingidx = bookingidx.sort_values(by=['second']) # sorting the records by seconds so that it is easier to plot time against acceleration_x
    bookingidx = bookingidx.reset_index(drop=True) # reindexing the dataframe for easier coding
    cleanedSetbookingidx = bookingidx.dropna()
    if len(cleanedSetbookingidx) < 5:
        set4_3fake = set4_3fake.drop(set4_3fake[set4_3fake.bookingID == bookingid].index)
        continue
    std = np.std(bookingidx['acceleration_x']) # finding the standard deviation of acceleration_x
    nulldf = bookingidx[bookingidx['acceleration_x'].isnull()] # make dataframe of records with acceleration_x being a null value
    for ind in nulldf.index: #retrieve the index of the row with acceleration_x = NULL
        id = nulldf[nulldf.index.values == ind]['ID'] # get the value of 'ID' column for that specific null record
        previousrowindex = ind - 1 # this is to retrieve the index of the previous row 
        afterrowindex = ind + 1 # this is to retrieve the index of the next row  
        valueofpreviousrow = bookingidx.iloc[previousrowindex, 4] # this is to retrieve the acceleration of the previous row
        valueofafterrow = bookingidx[bookingidx.index.values == afterrowindex]['acceleration_x'] # this is to retrieve the acceleration of the next row
        print(f'bookingid: {bookingid}, index: {int(ind)}, id: {int(id)}')
        if ind == 0: # if the null record is the first record of xbookingid
            if bookingidx[bookingidx.index.values == afterrowindex]['acceleration_x'].empty or bookingidx[bookingidx.index.values == afterrowindex]['acceleration_x'].isnull().bool(): # if the next record is also a null value, this is for records that are consecutive AT THE START OF THE BOOKINGID
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(bookingidx[bookingidx.index.values == afterrowindex+1]['acceleration_x']) - std -std # to get estimatedvalue of record 1, record 3- std -std = record 1
            else: # if then next record IS NOT a null value
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(valueofafterrow) - std # the estimated value of record 1 = record 2 - std
        
        elif ind ==  len(bookingidx)-1: # if the null record is THE LAST VALUE OF THE BOOKINGID
            print(f'the previous value is {valueofpreviousrow}') 
            estimatedvalue = valueofpreviousrow + std # the estimated value of the last record = second last record + std
        
        elif bookingidx[bookingidx.index.values == afterrowindex]['acceleration_x'].empty or bookingidx[bookingidx.index.values == afterrowindex]['acceleration_x'].isnull().bool(): # if there are 2 consecutive null values and arent the first or last record
            print(f'the after value is {float(valueofafterrow)}')
            estimatedvalue = valueofpreviousrow + std # estimated value = previous record + std
        
        else:
            print(f'the previous value is {valueofpreviousrow} and the after value is {float(valueofafterrow)}')
            estimatedvalue = np.median([valueofpreviousrow,float(valueofafterrow)]) # the estimated value = median of previous record and next record
        print(f'estimated value is {estimatedvalue}')
        realindex =  set4_3fake[set4_3fake['ID'] == int(id)].index.values # to get the index of the record IN THE SET
        bookingidx.at[int(ind),'acceleration_x'] = estimatedvalue # to assign estimated value to the corresponding record IN THE CURRENT BOOKING DATAFRAME THE CODE IS WORKING WITH NOW
        set4_3fake.at[int(realindex),'acceleration_x'] = estimatedvalue # this is to assign estimated value to the corresponding record IN THE SET
        ss = set4_3fake[set4_3fake.index.values == int(realindex)]['acceleration_x'] # just for printing to debug
        zz = bookingidx[bookingidx.index.values == int(realindex)]['acceleration_x'] # just for printing to debug
        print(f'the value back in set4_3fake is now {ss}')
        print(f'the value back in bookingidx is now {zz}')
        print('-----------------------')

bookingid: 1503238553777.0, index: 19, id: 3370201
the previous value is 0.1520782470703125 and the after value is 0.8046539306640625
estimated value is 0.4783660888671875
the value back in set4_3fake is now 85231    0.478366
Name: acceleration_x, dtype: float64
the value back in bookingidx is now Series([], Name: acceleration_x, dtype: float64)
-----------------------
bookingid: 1503238553777.0, index: 30, id: 7137338
the previous value is 0.20875244140625 and the after value is 0.581396484375
estimated value is 0.395074462890625
the value back in set4_3fake is now 179745    0.395074
Name: acceleration_x, dtype: float64
the value back in bookingidx is now Series([], Name: acceleration_x, dtype: float64)
-----------------------
bookingid: 1503238553777.0, index: 37, id: 6368319
the previous value is -0.0955535888671875 and the after value is 0.189013671875
estimated value is 0.04673004150390626
the value back in set4_3fake is now 160480    0.04673
Name: acceleration_x, dtype: float64
t

In [ ]:
set4_3fake.isnull().sum(axis = 0)

ID                   0
bookingID            0
Accuracy          1192
Bearing           1619
acceleration_x       0
acceleration_y    1724
acceleration_z     664
gyro_x            1350
gyro_y            1102
gyro_z            1361
second             956
Speed                0
dtype: int64

In [ ]:
# THIS IS ONLY FOR SET 3_3fake acceleration_y
justnullsdf = set4_3fake[set4_3fake['acceleration_y'].isnull()]
bookingids = justnullsdf['bookingID'].unique() # get all the bookingids that have null values inside them
for bookingid in bookingids: #loop for each bookingid so that we are dealing with the null values bookingid by bookingid which is easier to work with
    bookingidx = set4_3fake[set4_3fake['bookingID']==bookingid] # making a datfarame with bookingID = x
    bookingidx = bookingidx.sort_values(by=['second']) # sorting the records by seconds so that it is easier to plot time against acceleration_y
    bookingidx = bookingidx.reset_index(drop=True) # reindexing the dataframe for easier coding
    cleanedSetbookingidx = bookingidx.dropna()
    if len(cleanedSetbookingidx) < 5:
        set4_3fake = set4_3fake.drop(set4_3fake[set4_3fake.bookingID == bookingid].index)
        continue
    std = np.std(bookingidx['acceleration_y']) # finding the standard deviation of acceleration_y
    nulldf = bookingidx[bookingidx['acceleration_y'].isnull()] # make dataframe of records with acceleration_y being a null value
    for ind in nulldf.index: #retrieve the index of the row with acceleration_y = NULL
        id = nulldf[nulldf.index.values == ind]['ID'] # get the value of 'ID' column for that specific null record
        previousrowindex = ind - 1 # this is to retrieve the index of the previous row 
        afterrowindex = ind + 1 # this is to retrieve the index of the next row  
        valueofpreviousrow = bookingidx.iloc[previousrowindex, 5] # this is to retrieve the acceleration of the previous row
        valueofafterrow = bookingidx[bookingidx.index.values == afterrowindex]['acceleration_y'] # this is to retrieve the acceleration of the next row
        print(f'bookingid: {bookingid}, index: {int(ind)}, id: {int(id)}')
        if ind == 0: # if the null record is the first record of xbookingid
            if bookingidx[bookingidx.index.values == afterrowindex]['acceleration_y'].empty or bookingidx[bookingidx.index.values == afterrowindex]['acceleration_y'].isnull().bool(): # if the next record is also a null value, this is for records that are consecutive AT THE START OF THE BOOKINGID
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(bookingidx[bookingidx.index.values == afterrowindex+1]['acceleration_y']) - std -std # to get estimatedvalue of record 1, record 3- std -std = record 1
            else: # if then next record IS NOT a null value
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(valueofafterrow) - std # the estimated value of record 1 = record 2 - std
        
        elif ind ==  len(bookingidx)-1: # if the null record is THE LAST VALUE OF THE BOOKINGID
            print(f'the previous value is {valueofpreviousrow}') 
            estimatedvalue = valueofpreviousrow + std # the estimated value of the last record = second last record + std
        
        elif bookingidx[bookingidx.index.values == afterrowindex]['acceleration_y'].empty or bookingidx[bookingidx.index.values == afterrowindex]['acceleration_y'].isnull().bool(): # if there are 2 consecutive null values and arent the first or last record
            print(f'the after value is {float(valueofafterrow)}')
            estimatedvalue = valueofpreviousrow + std # estimated value = previous record + std
        
        else:
            print(f'the previous value is {valueofpreviousrow} and the after value is {float(valueofafterrow)}')
            estimatedvalue = np.median([valueofpreviousrow,float(valueofafterrow)]) # the estimated value = median of previous record and next record
        print(f'estimated value is {estimatedvalue}')
        realindex =  set4_3fake[set4_3fake['ID'] == int(id)].index.values # to get the index of the record IN THE SET
        bookingidx.at[int(ind),'acceleration_y'] = estimatedvalue # to assign estimated value to the corresponding record IN THE CURRENT BOOKING DATAFRAME THE CODE IS WORKING WITH NOW
        set4_3fake.at[int(realindex),'acceleration_y'] = estimatedvalue # this is to assign estimated value to the corresponding record IN THE SET
        ss = set4_3fake[set4_3fake.index.values == int(realindex)]['acceleration_y'] # just for printing to debug
        zz = bookingidx[bookingidx.index.values == int(realindex)]['acceleration_y'] # just for printing to debug
        print(f'the value back in set4_3fake is now {ss}')
        print(f'the value back in bookingidx is now {zz}')
        print('-----------------------')

bookingid: 1047972020402.0, index: 0, id: 3121
the after value is 9.811439
estimated value is 8.39163528537922
the value back in set4_3fake is now 69    8.391635
Name: acceleration_y, dtype: float64
the value back in bookingidx is now Series([], Name: acceleration_y, dtype: float64)
-----------------------
bookingid: 1511828488324.0, index: 15, id: 3005156
the previous value is -8.432192993164064 and the after value is -10.114773559570311
estimated value is -9.273483276367188
the value back in set4_3fake is now 76095   -9.273483
Name: acceleration_y, dtype: float64
the value back in bookingidx is now Series([], Name: acceleration_y, dtype: float64)
-----------------------
bookingid: 1511828488324.0, index: 26, id: 5446238
the previous value is -9.546685791015628 and the after value is -8.81485595703125
estimated value is -9.18077087402344
the value back in set4_3fake is now 136899   -9.180771
Name: acceleration_y, dtype: float64
the value back in bookingidx is now Series([], Name: acce

In [ ]:
set4_3fake.isnull().sum(axis = 0)

ID                   0
bookingID            0
Accuracy          1190
Bearing           1614
acceleration_x       0
acceleration_y       0
acceleration_z     663
gyro_x            1347
gyro_y            1100
gyro_z            1355
second             955
Speed                0
dtype: int64

In [ ]:
# THIS IS ONLY FOR SET 3_3fake acceleration_z
justnullsdf = set4_3fake[set4_3fake['acceleration_z'].isnull()]
bookingids = justnullsdf['bookingID'].unique() # get all the bookingids that have null values inside them
for bookingid in bookingids: #loop for each bookingid so that we are dealing with the null values bookingid by bookingid which is easier to work with
    bookingidx = set4_3fake[set4_3fake['bookingID']==bookingid] # making a datfarame with bookingID = x
    bookingidx = bookingidx.sort_values(by=['second']) # sorting the records by seconds so that it is easier to plot time against acceleration_z
    bookingidx = bookingidx.reset_index(drop=True) # reindexing the dataframe for easier coding
    cleanedSetbookingidx = bookingidx.dropna()
    if len(cleanedSetbookingidx) < 5:
        set4_3fake = set4_3fake.drop(set4_3fake[set4_3fake.bookingID == bookingid].index)
        continue
    std = np.std(bookingidx['acceleration_z']) # finding the standard deviation of acceleration_z
    nulldf = bookingidx[bookingidx['acceleration_z'].isnull()] # make dataframe of records with acceleration_z being a null value
    for ind in nulldf.index: #retrieve the index of the row with acceleration_z = NULL
        id = nulldf[nulldf.index.values == ind]['ID'] # get the value of 'ID' column for that specific null record
        previousrowindex = ind - 1 # this is to retrieve the index of the previous row 
        afterrowindex = ind + 1 # this is to retrieve the index of the next row  
        valueofpreviousrow = bookingidx.iloc[previousrowindex, 6] # this is to retrieve the acceleration of the previous row
        valueofafterrow = bookingidx[bookingidx.index.values == afterrowindex]['acceleration_z'] # this is to retrieve the acceleration of the next row
        print(f'bookingid: {bookingid}, index: {int(ind)}, id: {int(id)}')
        if ind == 0: # if the null record is the first record of xbookingid
            if bookingidx[bookingidx.index.values == afterrowindex]['acceleration_z'].empty or bookingidx[bookingidx.index.values == afterrowindex]['acceleration_z'].isnull().bool(): # if the next record is also a null value, this is for records that are consecutive AT THE START OF THE BOOKINGID
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(bookingidx[bookingidx.index.values == afterrowindex+1]['acceleration_z']) - std -std # to get estimatedvalue of record 1, record 3- std -std = record 1
            else: # if then next record IS NOT a null value
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(valueofafterrow) - std # the estimated value of record 1 = record 2 - std
        
        elif ind ==  len(bookingidx)-1: # if the null record is THE LAST VALUE OF THE BOOKINGID
            print(f'the previous value is {valueofpreviousrow}') 
            estimatedvalue = valueofpreviousrow + std # the estimated value of the last record = second last record + std
        
        elif bookingidx[bookingidx.index.values == afterrowindex]['acceleration_z'].empty or bookingidx[bookingidx.index.values == afterrowindex]['acceleration_z'].isnull().bool(): # if there are 2 consecutive null values and arent the first or last record
            print(f'the after value is {float(valueofafterrow)}')
            estimatedvalue = valueofpreviousrow + std # estimated value = previous record + std
        
        else:
            print(f'the previous value is {valueofpreviousrow} and the after value is {float(valueofafterrow)}')
            estimatedvalue = np.median([valueofpreviousrow,float(valueofafterrow)]) # the estimated value = median of previous record and next record
        print(f'estimated value is {estimatedvalue}')
        realindex =  set4_3fake[set4_3fake['ID'] == int(id)].index.values # to get the index of the record IN THE SET
        bookingidx.at[int(ind),'acceleration_z'] = estimatedvalue # to assign estimated value to the corresponding record IN THE CURRENT BOOKING DATAFRAME THE CODE IS WORKING WITH NOW
        set4_3fake.at[int(realindex),'acceleration_z'] = estimatedvalue # this is to assign estimated value to the corresponding record IN THE SET
        ss = set4_3fake[set4_3fake.index.values == int(realindex)]['acceleration_z'] # just for printing to debug
        zz = bookingidx[bookingidx.index.values == int(realindex)]['acceleration_z'] # just for printing to debug
        print(f'the value back in set4_3fake is now {ss}')
        print(f'the value back in bookingidx is now {zz}')
        print('-----------------------')

bookingid: 1666447310986.0, index: 11, id: 33428
the previous value is -3.2625793457031254 and the after value is -4.304248046875
estimated value is -3.7834136962890628
the value back in set4_3fake is now 801   -3.783414
Name: acceleration_z, dtype: float64
the value back in bookingidx is now Series([], Name: acceleration_z, dtype: float64)
-----------------------
bookingid: 1073741824033.0, index: 42, id: 40760
the previous value is -4.566983032226564 and the after value is -3.72105712890625
estimated value is -4.144020080566407
the value back in set4_3fake is now 986   -4.14402
Name: acceleration_z, dtype: float64
the value back in bookingidx is now Series([], Name: acceleration_z, dtype: float64)
-----------------------
bookingid: 1503238553604.0, index: 6, id: 78536
the previous value is 1.6812755 and the after value is 0.076298416
estimated value is 0.878786958
the value back in set4_3fake is now 1879    0.878787
Name: acceleration_z, dtype: float64
the value back in bookingidx is

In [ ]:
set4_3fake.isnull().sum(axis = 0)

ID                   0
bookingID            0
Accuracy          1190
Bearing           1610
acceleration_x       0
acceleration_y       0
acceleration_z       0
gyro_x            1347
gyro_y            1098
gyro_z            1354
second             955
Speed                0
dtype: int64

In [ ]:
# THIS IS ONLY FOR SET 3_3fake gyro_x
justnullsdf = set4_3fake[set4_3fake['gyro_x'].isnull()]
bookingids = justnullsdf['bookingID'].unique() # get all the bookingids that have null values inside them
for bookingid in bookingids: #loop for each bookingid so that we are dealing with the null values bookingid by bookingid which is easier to work with
    bookingidx = set4_3fake[set4_3fake['bookingID']==bookingid] # making a datfarame with bookingID = x
    bookingidx = bookingidx.sort_values(by=['second']) # sorting the records by seconds so that it is easier to plot time against gyro_x
    bookingidx = bookingidx.reset_index(drop=True) # reindexing the dataframe for easier coding
    cleanedSetbookingidx = bookingidx.dropna()
    if len(cleanedSetbookingidx) < 5:
        set4_3fake = set4_3fake.drop(set4_3fake[set4_3fake.bookingID == bookingid].index)
        continue
    std = np.std(bookingidx['gyro_x']) # finding the standard deviation of gyro_x
    nulldf = bookingidx[bookingidx['gyro_x'].isnull()] # make dataframe of records with gyro_x being a null value
    for ind in nulldf.index: #retrieve the index of the row with gyro_x = NULL
        id = nulldf[nulldf.index.values == ind]['ID'] # get the value of 'ID' column for that specific null record
        previousrowindex = ind - 1 # this is to retrieve the index of the previous row 
        afterrowindex = ind + 1 # this is to retrieve the index of the next row  
        valueofpreviousrow = bookingidx.iloc[previousrowindex, 7] # this is to retrieve the acceleration of the previous row
        valueofafterrow = bookingidx[bookingidx.index.values == afterrowindex]['gyro_x'] # this is to retrieve the acceleration of the next row
        print(f'bookingid: {bookingid}, index: {int(ind)}, id: {int(id)}')
        if ind == 0: # if the null record is the first record of xbookingid
            if bookingidx[bookingidx.index.values == afterrowindex]['gyro_x'].empty or bookingidx[bookingidx.index.values == afterrowindex]['gyro_x'].isnull().bool(): # if the next record is also a null value, this is for records that are consecutive AT THE START OF THE BOOKINGID
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(bookingidx[bookingidx.index.values == afterrowindex+1]['gyro_x']) - std -std # to get estimatedvalue of record 1, record 3- std -std = record 1
            else: # if then next record IS NOT a null value
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(valueofafterrow) - std # the estimated value of record 1 = record 2 - std
        
        elif ind ==  len(bookingidx)-1: # if the null record is THE LAST VALUE OF THE BOOKINGID
            print(f'the previous value is {valueofpreviousrow}') 
            estimatedvalue = valueofpreviousrow + std # the estimated value of the last record = second last record + std
        
        elif bookingidx[bookingidx.index.values == afterrowindex]['gyro_x'].empty or bookingidx[bookingidx.index.values == afterrowindex]['gyro_x'].isnull().bool(): # if there are 2 consecutive null values and arent the first or last record
            print(f'the after value is {float(valueofafterrow)}')
            estimatedvalue = valueofpreviousrow + std # estimated value = previous record + std
        
        else:
            print(f'the previous value is {valueofpreviousrow} and the after value is {float(valueofafterrow)}')
            estimatedvalue = np.median([valueofpreviousrow,float(valueofafterrow)]) # the estimated value = median of previous record and next record
        print(f'estimated value is {estimatedvalue}')
        realindex =  set4_3fake[set4_3fake['ID'] == int(id)].index.values # to get the index of the record IN THE SET
        bookingidx.at[int(ind),'gyro_x'] = estimatedvalue # to assign estimated value to the corresponding record IN THE CURRENT BOOKING DATAFRAME THE CODE IS WORKING WITH NOW
        set4_3fake.at[int(realindex),'gyro_x'] = estimatedvalue # this is to assign estimated value to the corresponding record IN THE SET
        ss = set4_3fake[set4_3fake.index.values == int(realindex)]['gyro_x'] # just for printing to debug
        zz = bookingidx[bookingidx.index.values == int(realindex)]['gyro_x'] # just for printing to debug
        print(f'the value back in set4_3fake is now {ss}')
        print(f'the value back in bookingidx is now {zz}')
        print('-----------------------')

bookingid: 1408749273120.0, index: 14, id: 5867981
the previous value is 0.0031280518 and the after value is 0.0107879639999999
estimated value is 0.00695800789999995
the value back in set4_3fake is now 147675    0.006958
Name: gyro_x, dtype: float64
the value back in bookingidx is now Series([], Name: gyro_x, dtype: float64)
-----------------------
bookingid: 1408749273120.0, index: 19, id: 26067
the previous value is -0.0075378419999999 and the after value is -0.0071411133
estimated value is -0.00733947764999995
the value back in set4_3fake is now 633   -0.007339
Name: gyro_x, dtype: float64
the value back in bookingidx is now Series([], Name: gyro_x, dtype: float64)
-----------------------
bookingid: 1408749273120.0, index: 25, id: 3461989
the previous value is -9.1552734e-05 and the after value is -0.016616821
estimated value is -0.008354186867
the value back in set4_3fake is now 87419   -0.008354
Name: gyro_x, dtype: float64
the value back in bookingidx is now Series([], Name: gyr

In [ ]:
set4_3fake.isnull().sum(axis = 0)

ID                   0
bookingID            0
Accuracy          1190
Bearing           1608
acceleration_x       0
acceleration_y       0
acceleration_z       0
gyro_x               0
gyro_y            1097
gyro_z            1352
second             953
Speed                0
dtype: int64

In [ ]:
# THIS IS ONLY FOR SET 3_3fake gyro_y
justnullsdf = set4_3fake[set4_3fake['gyro_y'].isnull()]
bookingids = justnullsdf['bookingID'].unique() # get all the bookingids that have null values inside them
for bookingid in bookingids: #loop for each bookingid so that we are dealing with the null values bookingid by bookingid which is easier to work with
    bookingidx = set4_3fake[set4_3fake['bookingID']==bookingid] # making a datfarame with bookingID = x
    bookingidx = bookingidx.sort_values(by=['second']) # sorting the records by seconds so that it is easier to plot time against gyro_y
    bookingidx = bookingidx.reset_index(drop=True) # reindexing the dataframe for easier coding
    cleanedSetbookingidx = bookingidx.dropna()
    if len(cleanedSetbookingidx) < 5:
        set4_3fake = set4_3fake.drop(set4_3fake[set4_3fake.bookingID == bookingid].index)
        continue
    std = np.std(bookingidx['gyro_y']) # finding the standard deviation of gyro_y
    nulldf = bookingidx[bookingidx['gyro_y'].isnull()] # make dataframe of records with gyro_y being a null value
    for ind in nulldf.index: #retrieve the index of the row with gyro_y = NULL
        id = nulldf[nulldf.index.values == ind]['ID'] # get the value of 'ID' column for that specific null record
        previousrowindex = ind - 1 # this is to retrieve the index of the previous row 
        afterrowindex = ind + 1 # this is to retrieve the index of the next row  
        valueofpreviousrow = bookingidx.iloc[previousrowindex, 8] # this is to retrieve the acceleration of the previous row
        valueofafterrow = bookingidx[bookingidx.index.values == afterrowindex]['gyro_y'] # this is to retrieve the acceleration of the next row
        print(f'bookingid: {bookingid}, index: {int(ind)}, id: {int(id)}')
        if ind == 0: # if the null record is the first record of xbookingid
            if bookingidx[bookingidx.index.values == afterrowindex]['gyro_y'].empty or bookingidx[bookingidx.index.values == afterrowindex]['gyro_y'].isnull().bool(): # if the next record is also a null value, this is for records that are consecutive AT THE START OF THE BOOKINGID
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(bookingidx[bookingidx.index.values == afterrowindex+1]['gyro_y']) - std -std # to get estimatedvalue of record 1, record 3- std -std = record 1
            else: # if then next record IS NOT a null value
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(valueofafterrow) - std # the estimated value of record 1 = record 2 - std
        
        elif ind ==  len(bookingidx)-1: # if the null record is THE LAST VALUE OF THE BOOKINGID
            print(f'the previous value is {valueofpreviousrow}') 
            estimatedvalue = valueofpreviousrow + std # the estimated value of the last record = second last record + std
        
        elif bookingidx[bookingidx.index.values == afterrowindex]['gyro_y'].empty or bookingidx[bookingidx.index.values == afterrowindex]['gyro_y'].isnull().bool(): # if there are 2 consecutive null values and arent the first or last record
            print(f'the after value is {float(valueofafterrow)}')
            estimatedvalue = valueofpreviousrow + std # estimated value = previous record + std
        
        else:
            print(f'the previous value is {valueofpreviousrow} and the after value is {float(valueofafterrow)}')
            estimatedvalue = np.median([valueofpreviousrow,float(valueofafterrow)]) # the estimated value = median of previous record and next record
        print(f'estimated value is {estimatedvalue}')
        realindex =  set4_3fake[set4_3fake['ID'] == int(id)].index.values # to get the index of the record IN THE SET
        bookingidx.at[int(ind),'gyro_y'] = estimatedvalue # to assign estimated value to the corresponding record IN THE CURRENT BOOKING DATAFRAME THE CODE IS WORKING WITH NOW
        set4_3fake.at[int(realindex),'gyro_y'] = estimatedvalue # this is to assign estimated value to the corresponding record IN THE SET
        ss = set4_3fake[set4_3fake.index.values == int(realindex)]['gyro_y'] # just for printing to debug
        zz = bookingidx[bookingidx.index.values == int(realindex)]['gyro_y'] # just for printing to debug
        print(f'the value back in set4_3fake is now {ss}')
        print(f'the value back in bookingidx is now {zz}')
        print('-----------------------')

bookingid: 1108101562489.0, index: 3, id: 310
the previous value is -0.0489973703674958 and the after value is 0.0686693423915841
estimated value is 0.009835986012044152
the value back in set4_3fake is now 11    0.009836
Name: gyro_y, dtype: float64
the value back in bookingidx is now 11    0.039996
Name: gyro_y, dtype: float64
-----------------------
bookingid: 1520418422970.0, index: 41, id: 319
the previous value is 0.0052557484112713 and the after value is -0.0042069955739981
estimated value is 0.0005243764186366002
the value back in set4_3fake is now 12    0.000524
Name: gyro_y, dtype: float64
the value back in bookingidx is now 12   -0.043949
Name: gyro_y, dtype: float64
-----------------------
bookingid: 1305670058109.0, index: 18, id: 2078
the previous value is -0.08357837 and the after value is 0.022120535
estimated value is -0.0307289175
the value back in set4_3fake is now 47   -0.030729
Name: gyro_y, dtype: float64
the value back in bookingidx is now 47    0.127338
Name: gyr

In [ ]:
set4_3fake.isnull().sum(axis = 0)

ID                   0
bookingID            0
Accuracy          1190
Bearing           1607
acceleration_x       0
acceleration_y       0
acceleration_z       0
gyro_x               0
gyro_y               0
gyro_z            1351
second             952
Speed                0
dtype: int64

In [ ]:
# THIS IS ONLY FOR SET 3_3fake gyro_z
justnullsdf = set4_3fake[set4_3fake['gyro_z'].isnull()]
bookingids = justnullsdf['bookingID'].unique() # get all the bookingids that have null values inside them
for bookingid in bookingids: #loop for each bookingid so that we are dealing with the null values bookingid by bookingid which is easier to work with
    bookingidx = set4_3fake[set4_3fake['bookingID']==bookingid] # making a datfarame with bookingID = x
    bookingidx = bookingidx.sort_values(by=['second']) # sorting the records by seconds so that it is easier to plot time against gyro_z
    bookingidx = bookingidx.reset_index(drop=True) # reindexing the dataframe for easier coding
    cleanedSetbookingidx = bookingidx.dropna()
    if len(cleanedSetbookingidx) < 5:
        set4_3fake = set4_3fake.drop(set4_3fake[set4_3fake.bookingID == bookingid].index)
        continue
    std = np.std(bookingidx['gyro_z']) # finding the standard deviation of gyro_z
    nulldf = bookingidx[bookingidx['gyro_z'].isnull()] # make dataframe of records with gyro_z being a null value
    for ind in nulldf.index: #retrieve the index of the row with gyro_z = NULL
        id = nulldf[nulldf.index.values == ind]['ID'] # get the value of 'ID' column for that specific null record
        previousrowindex = ind - 1 # this is to retrieve the index of the previous row 
        afterrowindex = ind + 1 # this is to retrieve the index of the next row  
        valueofpreviousrow = bookingidx.iloc[previousrowindex, 9] # this is to retrieve the acceleration of the previous row
        valueofafterrow = bookingidx[bookingidx.index.values == afterrowindex]['gyro_z'] # this is to retrieve the acceleration of the next row
        print(f'bookingid: {bookingid}, index: {int(ind)}, id: {int(id)}')
        if ind == 0: # if the null record is the first record of xbookingid
            if bookingidx[bookingidx.index.values == afterrowindex]['gyro_z'].empty or bookingidx[bookingidx.index.values == afterrowindex]['gyro_z'].isnull().bool(): # if the next record is also a null value, this is for records that are consecutive AT THE START OF THE BOOKINGID
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(bookingidx[bookingidx.index.values == afterrowindex+1]['gyro_z']) - std -std # to get estimatedvalue of record 1, record 3- std -std = record 1
            else: # if then next record IS NOT a null value
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(valueofafterrow) - std # the estimated value of record 1 = record 2 - std
        
        elif ind ==  len(bookingidx)-1: # if the null record is THE LAST VALUE OF THE BOOKINGID
            print(f'the previous value is {valueofpreviousrow}') 
            estimatedvalue = valueofpreviousrow + std # the estimated value of the last record = second last record + std
        
        elif bookingidx[bookingidx.index.values == afterrowindex]['gyro_z'].empty or bookingidx[bookingidx.index.values == afterrowindex]['gyro_z'].isnull().bool(): # if there are 2 consecutive null values and arent the first or last record
            print(f'the after value is {float(valueofafterrow)}')
            estimatedvalue = valueofpreviousrow + std # estimated value = previous record + std
        
        else:
            print(f'the previous value is {valueofpreviousrow} and the after value is {float(valueofafterrow)}')
            estimatedvalue = np.median([valueofpreviousrow,float(valueofafterrow)]) # the estimated value = median of previous record and next record
        print(f'estimated value is {estimatedvalue}')
        realindex =  set4_3fake[set4_3fake['ID'] == int(id)].index.values # to get the index of the record IN THE SET
        bookingidx.at[int(ind),'gyro_z'] = estimatedvalue # to assign estimated value to the corresponding record IN THE CURRENT BOOKING DATAFRAME THE CODE IS WORKING WITH NOW
        set4_3fake.at[int(realindex),'gyro_z'] = estimatedvalue # this is to assign estimated value to the corresponding record IN THE SET
        ss = set4_3fake[set4_3fake.index.values == int(realindex)]['gyro_z'] # just for printing to debug
        zz = bookingidx[bookingidx.index.values == int(realindex)]['gyro_z'] # just for printing to debug
        print(f'the value back in set4_3fake is now {ss}')
        print(f'the value back in bookingidx is now {zz}')
        print('-----------------------')

bookingid: 1683627180115.0, index: 1, id: 5751358
the previous value is -0.01909076 and the after value is 0.0065655797
estimated value is -0.006262590149999999
the value back in set4_3fake is now 144641   -0.006263
Name: gyro_z, dtype: float64
the value back in bookingidx is now Series([], Name: gyro_z, dtype: float64)
-----------------------
bookingid: 1683627180115.0, index: 44, id: 7363568
the previous value is 0.009009041 and the after value is 0.0065655797
estimated value is 0.00778731035
the value back in set4_3fake is now 185474    0.007787
Name: gyro_z, dtype: float64
the value back in bookingidx is now Series([], Name: gyro_z, dtype: float64)
-----------------------
bookingid: 1683627180115.0, index: 62, id: 6628394
the previous value is 0.0255024 and the after value is 0.012063367
estimated value is 0.0187828835
the value back in set4_3fake is now 167047    0.018783
Name: gyro_z, dtype: float64
the value back in bookingidx is now Series([], Name: gyro_z, dtype: float64)
----

In [ ]:
set4_3fake.isnull().sum(axis = 0)

ID                   0
bookingID            0
Accuracy          1190
Bearing           1606
acceleration_x       0
acceleration_y       0
acceleration_z       0
gyro_x               0
gyro_y               0
gyro_z               0
second             951
Speed                0
dtype: int64

In [ ]:
set4_3fake.to_csv('set4_3done.csv')